**Compute Validation Effectiveness for LLM with Retrieval-Augmented Generation**

In [ ]:
import pandas as pd
import json

baseline = "../data/evaluation/all_dependencies.csv"
results_file = "../data/evaluation/config1/all_dependencies_all_results.json"
model_name = "gpt-3.5-turbo-0125" # gpt-3.5-turbo-0125 gpt-4o-2024-05-13 llama3:8b llama3:70b

df_baseline = pd.read_csv(baseline)

with open(results_file, "r", encoding="utf-8") as src:
    data = json.load(src)


true_positives = []
true_negatives = []
false_positives = []
false_negatives = []
accuracy = []

for entry, (index, row) in zip(data, df_baseline.iterrows()):

    rating = row["final_rating"]
    try:
        response = entry["responses"][model_name]
        response_dict = json.loads(response)
        isDependency = response_dict["isDependency"]
    except json.JSONDecodeError as error:
        #print(index)
        #print(response_dict)
        #print(type(response_dict))
        try:
            isDependency = response_dict["isDependency"]
        except KeyError:
            continue
    except KeyError:
        continue

    
    if str(rating) == "Borderline":
        if isDependency:
            accuracy.append(1)
            true_positives.append(1)
        if not isDependency:
            accuracy.append(1)
            true_negatives.append(1)

    # TP: The LLM validates a dependency as correct and the dependency is correct
    if isDependency and str(rating).lower() == "true":
        accuracy.append(1)
        true_positives.append(1)
        
    # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
    if isDependency and str(rating).lower() == "false":
        accuracy.append(0)
        false_positives.append(1)

    # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
    if not isDependency and str(rating).lower() == "false":
        accuracy.append(1)
        true_negatives.append(1)

    # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
    if not isDependency and  str(rating).lower() == "true":
        accuracy.append(0)
        false_negatives.append(1)


print("Model: ", model_name)
print("File: ", results_file)
print("TP", sum(true_positives))
print("FP", sum(false_positives))
print("TN", sum(true_negatives))
print("FN", sum(false_negatives))
print("Sum correct response format: ", sum([sum(true_positives), sum(false_positives), sum(true_negatives), sum(false_negatives)]))

precision = sum(true_positives)/(sum(true_positives)+sum(false_positives))
recall = sum(true_positives)/(sum(true_positives)+sum(false_negatives))
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy", sum(accuracy)/len(accuracy))
print("Precision", precision)
print("Recall", recall)
print("F1 Score: ", f1_score)



**Compute Validation Effectiveness for LLM without Retrieval-Augmented Generation**

In [ ]:
import pandas as pd
import json

baseline = "../data/evaluation/all_dependencies.csv"
model_name = "gpt-3.5-turbo-0125" # gpt-3.5-turbo-0125 gpt-4o-2024-05-13 llama3:8b llama3:70b
results_file = f"../data/evaluation/without/all_dependencies_without_{model_name}.json"


df_baseline = pd.read_csv(baseline)

with open(results_file, "r", encoding="utf-8") as src:
    data = json.load(src)


true_positives = []
true_negatives = []
false_positives = []
false_negatives = []
accuracy = []

for entry, (index, row) in zip(data, df_baseline.iterrows()):

    rating = row["final_rating"]
    try:
        response = entry["response"]
        response_dict = json.loads(response)
        isDependency = response_dict["isDependency"]
    except json.JSONDecodeError as error:
        #print(index)
        #print(response_dict)
        #print(type(response_dict))
        try:
            isDependency = response_dict["isDependency"]
        except KeyError:
            continue
    except KeyError:
        continue

    
    if str(rating) == "Borderline":
        if isDependency:
            accuracy.append(1)
            true_positives.append(1)
        if not isDependency:
            accuracy.append(1)
            true_negatives.append(1)

    # TP: The LLM validates a dependency as correct and the dependency is correct
    if isDependency and str(rating).lower() == "true":
        accuracy.append(1)
        true_positives.append(1)
        
    # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
    if isDependency and str(rating).lower() == "false":
        accuracy.append(0)
        false_positives.append(1)

    # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
    if not isDependency and str(rating).lower() == "false":
        accuracy.append(1)
        true_negatives.append(1)

    # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
    if not isDependency and  str(rating).lower() == "true":
        accuracy.append(0)
        false_negatives.append(1)


print("Model: ", model_name)
print("File: ", results_file)
print("TP", sum(true_positives))
print("FP", sum(false_positives))
print("TN", sum(true_negatives))
print("FN", sum(false_negatives))
print("Sum correct response format: ", sum([sum(true_positives), sum(false_positives), sum(true_negatives), sum(false_negatives)]))

precision = sum(true_positives)/(sum(true_positives)+sum(false_positives))
recall = sum(true_positives)/(sum(true_positives)+sum(false_negatives))
f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy", sum(accuracy)/len(accuracy))
print("Precision", precision)
print("Recall", recall)
print("F1 Score: ", f1_score)

: 